In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np #대수학
import pandas as pd #전처리

import seaborn as sns #시각화
from matplotlib.patches import Patch
from matplotlib import pyplot as plt

from tensorflow.keras.models import load_model


plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

import warnings
%matplotlib inline

import configparser
from sklearn.preprocessing import MinMaxScaler

import re
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import datetime as dt

import os # 경로

import numpy as np
import sys
import random 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

import configparser
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.layers import Dropout
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gdp_re/training_data_0524.csv')
data = data.drop(['Unnamed: 0'], axis=1) # 변화율이 아닌 실제 데이터인 training_data_0524.csv를 사용하시려면 주석을 풀어주시면 됩니다.
data.tail()

,Y,Y1,Y2,Y3,YY1,YY2,YY3,MONTH,YEAR,QRT,...,X16_L0,X16_L1,X16_L2,X16_L3,X17_L1,X17_L2,X17_L3,X18_L1,X18_L2,X18_L3
72,-1.3,-3.2,2.2,1.1,['-1.3' '-3.2'],['-1.3' '-3.2' '2.2'],['-1.3' '-3.2' '2.2' '1.1'],1,2020,2020Q1,...,2119.01,2197.67,2087.96,2083.48,109.65,106.59,110.77,120.52,112.14,115.69
73,-3.2,2.2,1.1,1.7,['-3.2' '2.2'],['-3.2' '2.2' '1.1'],['-3.2' '2.2' '1.1' '1.7'],4,2020,2020Q2,...,1947.56,1754.64,1987.01,2119.01,110.41,97.41,99.07,116.45,102.29,118.39
74,2.2,1.1,1.7,0.8,['2.2' '1.1'],['2.2' '1.1' '1.7'],['2.2' '1.1' '1.7' '0.8'],7,2020,2020Q3,...,2249.37,2108.33,2029.60,1947.56,92.85,82.16,86.99,99.11,95.86,105.13
75,1.1,1.7,0.8,0.3,['1.1' '1.7'],['1.1' '1.7' '0.8'],['1.1' '1.7' '0.8' '0.3'],10,2020,2020Q4,...,2267.15,2327.89,2326.17,2249.37,113.80,94.52,100.72,107.69,98.61,107.70
76,1.7,0.8,0.3,1.2,['1.7' '0.8'],['1.7' '0.8' '0.3'],['1.7' '0.8' '0.3' '1.2'],1,2021,2021Q1,...,2976.21,2873.47,2591.34,2267.15,120.21,110.69,106.95,123.99,111.52,108.07


In [5]:
## 결측치를 확인 및 drop
data = data[data.iloc[:, 9:].apply(lambda row : sum(pd.isna(row)), axis = 1) == 0]
data = data.reset_index(drop = True)
data["Y"]

0     2.5
1     1.8
2     2.0
3     1.1
4    -0.7
     ... 
72   -1.3
73   -3.2
74    2.2
75    1.1
76    1.7
Name: Y, Length: 77, dtype: float64

In [6]:
## 코로나 미포함할 경우 주석을 풀어주시면 됩니다.
# data = data[(data['YEAR']!=2020)&(data['YEAR']!=2021)]
# data

In [7]:
data.set_index(['YEAR'], inplace=True)

In [8]:
train = data.drop(['Y','Y1','Y2','Y3', 'YY1','YY2','YY3','MONTH','QRT','TIME','STATE'], axis=1) ## training_data_0524.csv을 사용하실 때
# train = data.drop(['Y','YY1','YY2','YY3','MONTH','QRT','TIME','STATE'], axis=1) # train_data_n을 사용할 경우 주석을 풀어주시면 됩니다.

train.columns

Index(['X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2', 'X4_L3',
       'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1', 'X8_L2',
       'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2', 'X10_L3',
       'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2', 'X13_L3',
       'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1', 'X15_L2',
       'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1', 'X17_L2',
       'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3'],
      dtype='object')

In [9]:
## 주가 변수의 단위를 바꿔줄 때 사용하였습니다. 최근 시도에서는 사용하지 않았습니다.
# train["X7_L2"] = train["X7_L2"] / 100000 # 10 ** 6
# train["X7_L3"] = train["X7_L3"] / 100000 # 10 ** 6

In [10]:
## 표준화 (최근 시도에서는 사용하지 않았습니다.)
from sklearn.preprocessing import StandardScaler
# train = StandardScaler().fit_transform(train)
# pd.DataFrame(train)

In [11]:
## 정규화 (X에만 적용되었습니다.)
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
train

array([[0.10294118, 0.        , 0.58490566, ..., 0.        , 0.03074177,
        0.00414978],
       [0.77941176, 0.59210526, 0.67924528, ..., 0.021448  , 0.        ,
        0.        ],
       [1.        , 1.        , 0.8490566 , ..., 0.01977401, 0.06078937,
        0.03194364],
       ...,
       [0.10294118, 0.19736842, 0.        , ..., 0.71186441, 0.66283221,
        0.71125265],
       [0.25      , 0.23684211, 0.18867925, ..., 0.80163214, 0.69010313,
        0.73605482],
       [0.41176471, 0.36842105, 0.45283019, ..., 0.97216991, 0.81812773,
        0.73962555]])

In [12]:
df = pd.DataFrame(train, columns=['X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2',
       'X4_L3', 'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1',
       'X8_L2', 'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2',
       'X10_L3', 'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2',
       'X13_L3', 'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1',
       'X15_L2', 'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1',
       'X17_L2', 'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3'], index=data.index)
df["Y"] = data["Y"]/100 # Y in df is in pct-form.
df["Y"] = np.log1p(df["Y"]) # 로그변환
df["Y"]

YEAR
2002    0.024693
2002    0.017840
2002    0.019803
2002    0.010940
2003   -0.007025
          ...   
2020   -0.013085
2020   -0.032523
2020    0.021761
2020    0.010940
2021    0.016857
Name: Y, Length: 77, dtype: float64

In [13]:
## 시드 고정
def seed_everthing(seed):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  tf.random.set_seed(seed)

seed_everthing(1)

In [32]:
def lstm_model(X_train, y_train, X_test, y_test):
  model = Sequential()

  # model.add(Embedding(input_dim =X_train.shape[1], output_dim = 1))
  model.add(Embedding(input_dim =X_train.shape[1], output_dim = 1))

  # RNN 첫번째 은닉층 추가
  # model.add(LSTM(units=X_train.shape[1], activation='tanh', return_sequences = True))
  model.add(LSTM(units = 234, activation='tanh', return_sequences = True))
  # model.add(Dropout(0.6))
  model.add(Dropout(0.4))

  # RNN 두번째 은닉층 추가
  model.add(LSTM(int(234/2), activation='tanh', return_sequences = True))
  model.add(Dropout(0.4))

  # RNN 세번째 은닉층 추가
  model.add(LSTM(int(234/3), activation='tanh', return_sequences = True))
  model.add(Dropout(0.4))

  # RNN 네번째 은닉층 추가
  model.add(LSTM(10, activation='tanh', return_sequences = False))
  model.add(Dropout(0.4))

  # 출력층 추가
  model.add(Dense(units = 1))

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
  checkpoint = ModelCheckpoint(filepath = f"/content/drive/MyDrive/Colab Notebooks/gdp_re/model/model_{i}.h5", monitor='val_loss', save_best_only=True) #f"/content/drive/MyDrive/Colab Notebooks/gdp_re/model/moel_{i}.h5"
  early_stopping = EarlyStopping(monitor='val_loss', patience =60, mode='min')

  initial_learning_rate = 1e-4
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps = 100, decay_rate = 0.96, staircase=True)
  scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

  hist = model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=150,  batch_size = 8 ,callbacks=[checkpoint, early_stopping, scheduler])
  model_file = [fl for fl in os.listdir("/content/drive/MyDrive/Colab Notebooks/gdp_re/model") if re.findall(f"model_{i}", fl)][0] # 모델을 저장할 경로를 지정해주셔야 합니다.
  model = load_model(f"/content/drive/MyDrive/Colab Notebooks/gdp_re/model/{model_file}") # 모델을 불러올 경로를 지정해주셔야 합니다. 위의 경로와 같습니다.

  return model

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
from keras.backend import rnn
sample_size = len(df)
predicted_y = []

df_train = df[df.index < 2015] # 학습데이터
window = len(df_train) # 윈도우 사이즈 설정 (2002~2015)


df_test = df[df.index >= 2015] #검증데이터
forecast_horizon = len(df_test) # 검증데이터 길이

start_index = 0

# fit the model and make a forecast
for i in tqdm(range(forecast_horizon)): #검증데이터만큼 돌린다

    # writer = SummaryWriter()

    end_index = start_index + window 
    new_data = df.iloc[start_index : end_index] # df에서 end_index+1

    x_train, x_val, y_train, y_val = train_test_split(new_data.drop(["Y"], axis =1), new_data["Y"] ,test_size = 0.25, shuffle = False)

    # gets the trained neural network with given data
    model = lstm_model(x_train, y_train, x_val, y_val) #lstm_model함수는 hist를 return해줌

    x_new = df.iloc[end_index]
    x_new = pd.DataFrame(x_new).T.drop(["Y"], axis = 1)
    y_pred = model.predict(x_new)
    # y_pred = y_pred[0][0]    

    predicted_y.append(y_pred)
    print(y_pred)

    start_index = start_index + 1 # for rolling-window forecasts

    # window = window + 1 # for recursive forecasts
   

  0%|          | 0/25 [00:00<?, ?it/s]


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 7s 430ms/step - loss: 1.8442e-04 - root_mean_squared_error: 0.0136 - val_loss: 5.0702e-05 - val_root_mean_squared_error: 0.0071 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 0s 44ms/step - loss: 1.4228e-04 - root_mean_squared_error: 0.0119 - val_loss: 1.4175e-05 - val_root_mean_squared_error: 0.0038 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 0s 43ms/step - loss: 1.2853e-04 - root_mean_squared_error: 0.0113 - val_loss: 6.3933e-06 - val_root_mean_squared_error: 0.0025 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 0s 26ms/step - loss: 1.2880e-04 - root_mean

  4%|▍         | 1/25 [00:19<07:41, 19.24s/it]

[[0.0065142]]

Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 7s 456ms/step - loss: 1.4401e-04 - root_mean_squared_error: 0.0120 - val_loss: 5.4898e-05 - val_root_mean_squared_error: 0.0074 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 0s 44ms/step - loss: 1.4497e-04 - root_mean_squared_error: 0.0120 - val_loss: 2.1716e-05 - val_root_mean_squared_error: 0.0047 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 0s 45ms/step - loss: 1.2319e-04 - root_mean_squared_error: 0.0111 - val_loss: 5.8187e-06 - val_root_mean_squared_error: 0.0024 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 0s 28ms/step - loss: 1.2010e-

  8%|▊         | 2/25 [00:37<07:06, 18.56s/it]

...

[[0.00355412]]

Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 7s 426ms/step - loss: 1.0991e-04 - root_mean_squared_error: 0.0105 - val_loss: 2.4420e-04 - val_root_mean_squared_error: 0.0156 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 0s 49ms/step - loss: 9.9504e-05 - root_mean_squared_error: 0.0100 - val_loss: 1.7939e-04 - val_root_mean_squared_error: 0.0134 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 0s 29ms/step - loss: 9.6314e-05 - root_mean_squared_error: 0.0098 - val_loss: 1.9156e-04 - val_root_mean_squared_error: 0.0138 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 0s 29ms/step - loss: 8.1765e

 96%|█████████▌| 24/25 [11:59<00:29, 29.15s/it]

[[0.00357859]]

Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 7s 377ms/step - loss: 1.1692e-04 - root_mean_squared_error: 0.0108 - val_loss: 2.3762e-04 - val_root_mean_squared_error: 0.0154 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 0s 48ms/step - loss: 8.7510e-05 - root_mean_squared_error: 0.0094 - val_loss: 1.9040e-04 - val_root_mean_squared_error: 0.0138 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 0s 48ms/step - loss: 8.6941e-05 - root_mean_squared_error: 0.0093 - val_loss: 1.7464e-04 - val_root_mean_squared_error: 0.0132 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 0s 29ms/step - loss: 9.0967e

100%|██████████| 25/25 [12:20<00:00, 29.61s/it]

[[0.00437879]]


In [35]:
# 예측된 성장률 (추정한 현재 GDP - 실제 전년도 GDP)

In [36]:
predicted_y

[array([[0.0065142]], dtype=float32),
 array([[0.00727677]], dtype=float32),
 array([[0.0032409]], dtype=float32),
 array([[0.0078816]], dtype=float32),
 array([[0.00773489]], dtype=float32),
 array([[0.00768952]], dtype=float32),
 array([[0.00755786]], dtype=float32),
 array([[0.00710123]], dtype=float32),
 array([[0.00697747]], dtype=float32),
 array([[0.00708141]], dtype=float32),
 array([[0.00695837]], dtype=float32),
 array([[0.00746713]], dtype=float32),
 array([[0.00722638]], dtype=float32),
 array([[0.00636033]], dtype=float32),
 array([[0.0072733]], dtype=float32),
 array([[0.00618816]], dtype=float32),
 array([[0.00710363]], dtype=float32),
 array([[0.00641067]], dtype=float32),
 array([[0.00694916]], dtype=float32),
 array([[0.00676388]], dtype=float32),
 array([[0.00690266]], dtype=float32),
 array([[0.00544373]], dtype=float32),
 array([[0.00355412]], dtype=float32),
 array([[0.00357859]], dtype=float32),
 array([[0.00437879]], dtype=float32)]

- layer개수(2,3,4,5,6), unit(Int), batch(Int)
- 새로운 시점의 데이터가 들어왔을 때

In [37]:
predicted_y = [v[0][0] for v in predicted_y]

In [38]:
df_test["loss"] = predicted_y - df_test["Y"]
df_test["predicted_y"] = predicted_y
df_test["real_y"] = data[data.index >= 2015]['Y'] # 실제값

In [39]:
df_test["Y_exp"] = (np.expm1(df_test["Y"]))*100 # 로그변환했던 실제 Y값 (지수변환*100)
df_test['predicted_y_exp'] = (np.expm1(df_test["predicted_y"]))*100 # 예측값(지수변환*100)

# df_test["predicted_y_r"] = round(df_test["predicted_y"], 1) #소수점 변환

In [40]:
df_test[["real_y", "Y_exp", "predicted_y_exp", "loss"]]#.to_csv("./prediction_result.csv", sep = ",", encoding = "cp949")

,real_y,Y_exp,predicted_y_exp,loss
YEAR,,,,
2015,0.8,0.8,0.653547,-0.001454
2015,0.4,0.4,0.730331,0.003285
2015,1.5,1.5,0.324616,-0.011648
2015,0.7,0.7,0.791274,0.000906
2016,0.3,0.3,0.776488,0.004739
2016,1.1,1.1,0.771916,-0.003250
2016,0.5,0.5,0.758649,0.002570
2016,0.6,0.6,0.712650,0.001119
2017,1.0,1.0,0.700187,-0.002973


In [41]:
## 코로나 포함
print((np.sum((df_test['predicted_y_exp'] - df_test["real_y"]) ** 2) / len(df_test['predicted_y_exp'])) ** 0.5)

1.0755776322804507


In [42]:
## 코로나 제외
drop_covid = df_test[["Y_exp", "predicted_y_exp", "loss"]].drop(labels=2020, axis=0)
print((np.sum((drop_covid['predicted_y_exp'] - drop_covid["Y_exp"]) ** 2) / len(drop_covid['predicted_y_exp'])) ** 0.5)

0.5760938861124442


In [ ]:
## 그래프

#  y_vloss = hist.history['loss']
# y_loss = hist.history['val_loss']

# x_len = np.arange(len(y_loss))
# plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
# plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

# plt.legend(loc='upper right')
# plt.grid()
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.show()

In [ ]:
# ## 교차검증 시 데이터 분할 시각화 helper function 가져오기
# from sklearn.model_selection import TimeSeriesSplit 
# n_split = 4
# tscv = TimeSeriesSplit(n_splits=n_split)
# X = X.set_index(data["datetime"])

# for fold, (train_index, test_index) in enumerate(tscv.split(X)):
#   print("Fold: {}".format(fold))
#   print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
#   print("\n")
#   X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#   y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  